# Internet Computer

## Setup
* Install [dfx](https://internetcomputer.org/docs/building-apps/getting-started/install#installing-dfx-via-dfxvm)
* Checkout the `basic_solana` example
    ```
    git clone git@github.com:dfinity/sol-rpc-canister.git
    cd sol-rpc-canister/examples/basic_solana
    ```

In [53]:
!dfx --version

dfx 0.26.1


In [54]:
!pwd

/Users/greg/git/sol-rpc-canister/examples/basic_solana


## Deploy

Deploy `basic_solana` example on mainnet.

In [55]:
!dfx identity use admin

Using identity: "admin".


In [56]:
!dfx deploy --ic basic_solana

Deploying: basic_solana
All canisters have already been created.
Building canister 'basic_solana'.                                        
Executing 'cargo build --no-default-features --target wasm32-unknown-unknown --release'
ilding canisters...                                                             Finished `release` profile [optimized] target(s) in 1.47s
⠙ Installing canisters...                                                       
⠙ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai                                                                        
⠚ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai                                                                        
⠚ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai                                                                        
⠒ Installing canisters...ic_solana, with canister ID vond4-2qaaa-aaaar-qbmiq-cai               

In [57]:
!dfx canister status --ic --all

Canister status call result for basic_solana.
Status: Running
Controllers: 4nct6-qlari-qhlgv-qkhdd-5hak6-gtdms-ra2ya-gi7qq-2lref-zg34x-bqe ezu3d-2mifu-k3bh4-oqhrj-mbrql-5p67r-pp6pr-dbfra-unkx5-sxdtv-rae mf7xa-laaaa-aaaar-qaaaa-cai
Memory allocation: 0 Bytes
Compute allocation: 0 %
Freezing threshold: 2_592_000 Seconds
Idle cycles burned per day: 67_686_185 Cycles
Memory Size: 2_532_485 Bytes
Balance: 6_890_311_679_322 Cycles
Reserved: 0 Cycles
Reserved cycles limit: 5_000_000_000_000 Cycles
Wasm memory limit: 3_221_225_472 Bytes
Wasm memory threshold: 0 Bytes
Module hash: 0x4ae0c236e7b5bec153f527c6b291088af48e202d5eb154f8d9423e0005bcf160
Number of queries: 0
Instructions spent in queries: 0
Total query request payload size: 0 Bytes
Total query response payload size: 0 Bytes
Log visibility: controllers


## Create a new identity
Create a new identity, which is not a controller of the `basic_solana` canister, to interact with it.

In [ ]:
!dfx identity new helium_demo

In [ ]:
!dfx identity use helium_demo

In [ ]:
!dfx identity get-principal

In [ ]:
%env MY_PRINCIPAL="hopjo-r5uca-qhtpi-owytc-7loj3-h2xel-wxdqf-htpwz-7gxq7-rqdlc-vae"

## Read Balance

In [ ]:
%env SOLANA_CLI_ADDRESS="DTXBMjo6tcosChHeV83Aq6gZuH8K7JYA6EL4ADaMi3Zc"

In [ ]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic


### Explanations

* SOL has 9 decimals: 1 SOL == `1_000_000_000` Lamports
* [Logs of SOL RPC canister](https://titvo-eiaaa-aaaar-qaogq-cai.raw.icp0.io/logs?sort=desc)
    * 3 [HTTPS outcalls](https://support.dfinity.org/hc/en-us/articles/12342341722260-What-is-an-HTTPS-Outcall):
        * `solana-devnet.g.alchemy.com`
        * `lb.drpc.org`
        * `devnet.helius-rpc.com`
    * each HTTPS outcall involves each replica in the subnet making an HTTPS request => 34 HTTPs request for the [fiduciary subnet](https://dashboard.internetcomputer.org/subnet/pzp6e-ekpqk-3c5x7-2h6so-njoeq-mt45d-h3h6c-q3mxf-vpeq5-fk5o7-yae)
    *  TOTAL: 102 HTTPS requests distributed all-over the world! 💪
    * **no single point of failure** 🛡️



*Request airdrop with Solana CLI*


In [ ]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic


## Send SOL

* Canister can control an Ed25519 public/private key pair thanks to [threshold Schnorr](https://internetcomputer.org/docs/building-apps/network-features/signatures/t-schnorr/).
    * Each replica holds a share of a secret key
    * Each replica can use its share to create a signature share for a given message
    * Combining enough signature shares result in a standard signature for that message
    * ℹ️ At no moment is the secret key reconstructed.

### Fund address

In [ ]:
!dfx canister call basic_solana solana_account --ic

In [ ]:
%env SOLANA_CANISTER_ADDRESS="G7Tgmmi5y4KMByBULWnu6JHaukszKVQRgqCgVy8bKbPX"

In [ ]:
!dfx canister call basic_solana get_balance --ic

*Fund address via Solana CLI*

In [ ]:
!dfx canister call basic_solana get_balance --ic


### Send transaction

Send transaction of 1 Lamport back to Solana CLI address

In [ ]:
!dfx canister call basic_solana send_sol "(opt principal $MY_PRINCIPAL, $SOLANA_CLI_ADDRESS, 1 : nat)" --ic

Check transaction on [Solana explorer](https://explorer.solana.com/?cluster=devnet):
* nothing distinguishes a transaction signed via threshold Ed25529 from a regular transaction
* address controlled by the canister also paid the transaction fee

In [ ]:
!dfx canister call basic_solana get_balance --ic


* 1 major difficulty is getting a recent blockhash that must be part of the transaction.
* See the [code](https://github.com/dfinity/sol-rpc-canister/blob/8e7e32f50a2e05fa671a3df8f7b19f64c5abbdce/examples/basic_solana/src/main.rs#L226) for more details.

## Send USDC

Send 1 USDC from addressed controlled by canister to my Solana CLI wallet

### Fund address controlled by canister

* USDC devnet: [4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU](https://explorer.solana.com/address/4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU?cluster=devnet)
* USDC [faucet](https://faucet.circle.com/)

In [ ]:
%env USDC_DEVNET="4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU"

In [ ]:
!dfx canister call basic_solana associated_token_account "(opt principal $MY_PRINCIPAL, $USDC_DEVNET)" --ic


In [ ]:
!dfx canister call basic_solana get_spl_token_balance "(null, $USDC_DEVNET)" --ic


In [ ]:
!dfx canister call basic_solana send_spl_token "(opt principal $MY_PRINCIPAL, $USDC_DEVNET, $SOLANA_CLI_ADDRESS, 1_000_000 : nat)" --ic